In [27]:
import numpy as np
import json
from tournament import Tournament


with open('teams.json', 'r') as f:
    teams = json.load(f)

with open('locations.json', 'r') as f:
    locations = json.load(f)

stadiums = {}
for loc in locations:
    snames = []
    for s in locations[loc]['stadiums']:
        snames.append(s)
        stadiums[s] = {
            'location': loc,
            'size': locations[loc]['stadiums'][s]['size'] 
        }
    
    locations[loc]['stadiums'] = snames

timeslot_values = [100,130,50,60,110,50,40] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening']
timeslots = [{'value': v, 'name': n} for (n,v) in zip(timeslot_names, timeslot_values)]
tourn = Tournament(teams = teams, locations = locations, stadiums=stadiums, timeslots = timeslots, rounds = 22)


In [28]:
tourn.fixture_matrix.shape

(18, 18, 9, 7, 22)

In [48]:
def find_best_inround(attractiveness_matrix, round = None):
    if round is not None:
        x = attractiveness_matrix[:,:,:,:,round]
    else:
        x = attractiveness_matrix
    return tuple(np.unravel_index(np.argmax(x), x.shape))

def find_k_best_inround(attractiveness_matrix, k, round = None, ):
    if round is not None:
        x = attractiveness_matrix[:,:,:,:,round]
    else:
        x = attractiveness_matrix
    flat = x.flatten()

    # Find the indices of the N largest values in the flattened array
    indices = np.argpartition(flat, -k)[-k:]
    # Convert the flattened indices back to the indices in the original matrix
    return [tuple(np.unravel_index(idx, x.shape)) for idx in indices]

# deletes a pair of teams (order matters)
def delete_team_pair(matrix, i, j):
    # todo, might not be worth it compared to setting attractiveness to -1


    pass

def construct_greedy_round(attractiveness_matrix, round, timeslots, track_games = False,
                            fixture_matrix = None):
    games = [None for _ in range(timeslots)]
    games_fulfilled = 0
    tmp_matrix = attractiveness_matrix[:,:,:,:,round]

    # matrix to return to keep track of played games between two teams
    if track_games: return_matrix = attractiveness_matrix
    while games_fulfilled < timeslots:
        new_game = find_best_inround(tmp_matrix)
        games_fulfilled += 1
        
        team1, team2, stadium, timeslot = new_game
        tourn.print_game(team1, team2, stadium, round, timeslot)
        games[timeslot] = [team1,team2,stadium]
        
        # delete the timeslot and games played between the two teams for future considerations
        # this is done by setting the attractiveness to -1
        tmp_matrix[:,:,:,timeslot] = -1
        tmp_matrix[team1, team2, :, :] = -1

        # if we are running this in a loop to construct the whole schedule we want to keep track of the games played
        # between teams across all rounds
        if track_games: return_matrix[team1,team2,:,:,:] = -1
        if fixture_matrix is not None: fixture_matrix[team1, team2, stadium, timeslot, round] = 1
    
    if track_games:
        if fixture_matrix is not None:
            return games, return_matrix, fixture_matrix
        return games, return_matrix
    if fixture_matrix is not None:
        return games, fixture_matrix
    return games

def construct_greedy_schedule(attractiveness_matrix, rounds, timeslots):
    tmp_matrix = attractiveness_matrix.copy()
    fixture = np.zeros(shape = attractiveness_matrix.shape)
    schedule = [None for _ in range(rounds)]
    for round in range(rounds):
        print(f"Round {round}: ")
        games, tmp_matrix, fixture = construct_greedy_round(tmp_matrix, round, timeslots, 
                                                            track_games=True,
                                                            fixture_matrix=fixture)
        schedule[round] = games
    
    return schedule, fixture


def construct_greedy_round_random(attractiveness_matrix, round, timeslots,
                                    rcl_length = 10,
                                    track_games = False,
                                    fixture_matrix = None):
    games = [None for _ in range(timeslots)]
    games_fulfilled = 0
    tmp_matrix = attractiveness_matrix[:,:,:,:,round]

    # matrix to return to keep track of played games between two teams
    if track_games: return_matrix = attractiveness_matrix
    while games_fulfilled < timeslots:
        candidate_list = find_k_best_inround(tmp_matrix, k = rcl_length)

        # select at random
        new_game = candidate_list[np.random.choice(rcl_length)]
        
        
        games_fulfilled += 1
        
        team1, team2, stadium, timeslot = new_game
        tourn.print_game(team1, team2, stadium, round, timeslot)
        games[timeslot] = [team1,team2,stadium]
        
        # delete the timeslot and games played between the two teams for future considerations
        # this is done by setting the attractiveness to -1
        tmp_matrix[:,:,:,timeslot] = -1
        tmp_matrix[team1, team2, :, :] = -1

        # if we are running this in a loop to construct the whole schedule we want to keep track of the games played
        # between teams across all rounds
        if track_games: return_matrix[team1,team2,:,:,:] = -1
        if fixture_matrix is not None: fixture_matrix[team1, team2, stadium, timeslot, round] = 1
    
    if track_games:
        if fixture_matrix is not None:
            return games, return_matrix, fixture_matrix
        return games, return_matrix
    if fixture_matrix is not None:
        return games, fixture_matrix
    return games
    

def construct_greedy_schedule_random(attractiveness_matrix, rounds, timeslots, rcl_length = 10):
    tmp_matrix = attractiveness_matrix.copy()
    fixture = np.zeros(shape = attractiveness_matrix.shape)
    schedule = [None for _ in range(rounds)]
    for round in range(rounds):
        print(f"Round {round}: ")
        games, tmp_matrix, fixture = construct_greedy_round_random(tmp_matrix, round, timeslots, 
                                                                    rcl_length=rcl_length,
                                                                    track_games=True,
                                                                    fixture_matrix=fixture)
        schedule[round] = games
    
    return schedule, fixture



In [37]:
arr = tourn.attractiveness_matrix
result, fixture = construct_greedy_schedule(arr, rounds = 22, timeslots = 7)

Round 0: 
Friday Night: Collingwood Magpies vs. Collingwood Magpies at MCG
Saturday Night: Brisbane Lions vs. Collingwood Magpies at MCG
Thursday Night: Collingwood Magpies vs. Brisbane Lions at MCG
Saturday Evening: Port Adelaide Power vs. Collingwood Magpies at MCG
Saturday Afternoon: Collingwood Magpies vs. Melbourne Demons at MCG
Sunday Afternoon: Melbourne Demons vs. Collingwood Magpies at MCG
Sunday Evening: Collingwood Magpies vs. Port Adelaide Power at Adelaide Oval
Round 1: 
Friday Night: Carlton Blues vs. Carlton Blues at MCG
Saturday Night: Port Adelaide Power vs. Melbourne Demons at MCG
Thursday Night: Carlton Blues vs. Melbourne Demons at MCG
Saturday Evening: Melbourne Demons vs. Carlton Blues at MCG
Saturday Afternoon: Melbourne Demons vs. Melbourne Demons at MCG
Sunday Afternoon: Carlton Blues vs. Collingwood Magpies at MCG
Sunday Evening: Collingwood Magpies vs. Carlton Blues at MCG
Round 2: 
Friday Night: Geelong Cats vs. Essendon Bombers at MCG
Saturday Night: St Kil

In [38]:
fixture.shape

(18, 18, 9, 7, 22)

In [39]:
tourn.fixture_attractiveness(fixture=fixture)

1540000.0

In [40]:
tourn.feasibility(fixture=fixture)

(359.0, 47.0)

In [49]:
arr = tourn.attractiveness_matrix
result, fixture = construct_greedy_schedule_random(arr, rounds = 22, timeslots = 7)

Round 0: 
Thursday Night: Collingwood Magpies vs. Collingwood Magpies at MCG
Saturday Night: Collingwood Magpies vs. Brisbane Lions at MCG
Saturday Evening: Brisbane Lions vs. Collingwood Magpies at MCG
Friday Night: Port Adelaide Power vs. Collingwood Magpies at Marvel
Sunday Afternoon: Melbourne Demons vs. Collingwood Magpies at MCG
Saturday Afternoon: Collingwood Magpies vs. Melbourne Demons at MCG
Sunday Evening: Carlton Blues vs. Collingwood Magpies at MCG
Round 1: 
Saturday Night: Collingwood Magpies vs. Port Adelaide Power at Adelaide Oval
Friday Night: Geelong Cats vs. Essendon Bombers at MCG
Thursday Night: Collingwood Magpies vs. Carlton Blues at MCG
Saturday Afternoon: Melbourne Demons vs. Carlton Blues at MCG
Sunday Evening: Carlton Blues vs. Carlton Blues at MCG
Sunday Afternoon: St Kilda Saints vs. Carlton Blues at MCG
Saturday Evening: Carlton Blues vs. Melbourne Demons at MCG
Round 2: 
Friday Night: Port Adelaide Power vs. Melbourne Demons at MCG
Thursday Night: Melbour

In [50]:
tourn.fixture_attractiveness(fixture=fixture)

1540000.0

In [51]:
tourn.feasibility(fixture=fixture)

(362.0, 49.0)